## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import os
import numpy as np

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [ ]:
from functools import partialmethod
pd.DataFrame.head = partialmethod(pd.DataFrame.head, n=1)

In [ ]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

In [ ]:
#clean duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#fix date and date time
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['timestamp'].dt.date)
df['hour'] = df['timestamp'].dt.strftime('%H')
df['hours_Min'] = df['timestamp'].dt.strftime('%H:%M')

# this cell is used to convert T/F features into 0/1 features
df['Bump'] = df['Bump'].astype(int)
df['Crossing'] = df['Crossing'].astype(int)
df['Give_Way'] = df['Give_Way'].astype(int)
df['Junction'] = df['Junction'].astype(int)
df['No_Exit'] = df['No_Exit'].astype(int)
df['Railway'] = df['Railway'].astype(int)
df['Roundabout'] = df['Roundabout'].astype(int)
df['Stop'] = df['Stop'].astype(int)
df['Amenity'] = df['Amenity'].astype(int)
df['hour'] = df['hour'].astype(int)

# this cell is used to convert the Side column from L/R data into 0/1 data
df['Side'] = df['Side'].map(dict(L=0, R=1))

In [ ]:
#show unique Value
oneValuedcols = []
for col in df.columns:    
    if len(df[col].unique()) <10:
        print(f"Col name =" ,col)
        print(df[col].unique())
        print("______")
    if len(df[col].unique()) == 1:
        oneValuedcols.append(col)
df.drop(oneValuedcols,axis=1,inplace=True)

In [ ]:
df['Severity'].value_counts()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20,10))
for (serv, ax) in zip(np.arange(1,5), axes.flatten()):
    df[[ 'hour' , "Severity" ]].loc[df['Severity'] == serv].groupby(['hour']).agg(['count']).plot(kind = "bar" ,ax =ax)    
    ax.legend(f"count")
    ax.set_title(f"Severity {str(serv)} Count vs Hours of the day")


In [ ]:
df[[ 'hour' , "Severity" ]].loc[df['Severity'] == 1].groupby(['hour']).agg(['count']).plot(kind = "bar")
df[[ 'hour' , "Severity" ]].loc[df['Severity'] == 2].groupby(['hour']).agg(['count']).plot(kind = "bar")
df[[ 'hour' , "Severity" ]].loc[df['Severity'] == 3].groupby(['hour']).agg(['count']).plot(kind = "bar")
df[[ 'hour' , "Severity" ]].loc[df['Severity'] == 4].groupby(['hour']).agg(['count']).plot(kind = "bar")

In [ ]:
plt.scatter(x = df['hour'] , y = df['Severity'] , alpha = .2)

We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [ ]:
df.isnull().sum(axis = 0)

### reading holidays xml

In [ ]:
import xml.etree.ElementTree as ET

cols = ["date", "description"]
rows = []

#Parse XML file

tree = ET.parse(os.path.join(dataset_path, 'holidays.xml'))
root = tree.getroot()
for elem in root:
        date = elem.find("date").text
        description = elem.find("description").text
        rows.append({"date": date, 
                  "description": description})

holidays_df = pd.DataFrame(rows, columns = cols) 

print("The shape of the dataset is {}.\n\n".format(holidays_df.shape))
holidays_df.head()

In [ ]:
holidays_df['date'] = pd.to_datetime(holidays_df['date'])

In [ ]:
holidays_df.drop_duplicates(inplace= True)

In [ ]:
holidays_df.isnull().sum(axis = 0)

### reading weather csv

In [ ]:
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))
print("The shape of the dataset is {}.\n\n".format(weather_df.shape))
weather_df.head()

In [ ]:
weather_df.drop_duplicates(inplace=True)
weather_df['date'] = pd.to_datetime(weather_df[['Year', 'Month','Day']])
weather_df.drop(['Year' , 'Day' , 'Month'], inplace=True , axis  =1 )
weather_df = weather_df.rename(columns={'Hour': 'hour',})
weather_df.head()

In [ ]:
weather_df.isnull().sum(axis = 0)

In [ ]:
weather_df  = weather_df[weather_df.Weather_Condition.notnull()]

In [ ]:
weather_df.isnull().sum(axis = 0)

In [ ]:
weather_df  = weather_df[weather_df['Temperature(F)'].notnull()]

In [ ]:
344/(weather_df.shape[0])*100

In [ ]:
weather_df.isnull().sum(axis = 0)

In [ ]:
weather_df.describe()

In [ ]:
weather_df['Precipitation(in)'].hist();
weather_df[weather_df['Precipitation(in)'].isnull()]

In [ ]:
weather_df['Wind_Speed(mph)'].hist();
weather_df[weather_df['Wind_Speed(mph)'].isnull()]

### Merging the Data into one DataFrame

In [ ]:
df.head()

In [ ]:
weather_df.head()

In [ ]:
holidays_df.head()

In [ ]:
weather_df.drop_duplicates(subset=['date','hour'],inplace=True)

In [ ]:
merged = pd.merge(df,weather_df,how = "left" , left_on=['date','hour'],right_on=['date','hour'] )
merged.info()

In [ ]:
merged.isnull().sum(axis = 0)/merged.shape[0]*100

In [ ]:
merged  = merged[merged.Weather_Condition.notnull()]
merged.isnull().sum(axis = 0)/merged.shape[0]*100

In [ ]:
merged = merged.merge(holidays_df , how = 'left' , on = 'date')

In [ ]:
merged.columns

In [ ]:
merged.info()

In [ ]:
holidays_df

In [ ]:
merged.info()

In [ ]:
merged['description'] = merged['description'].fillna(0)
merged['description'].unique()

In [ ]:
merged.loc[merged['description']!= 0, 'description'] = 1

In [ ]:
merged['description'].unique()

In [ ]:
merged.info()

In [ ]:
merged.isnull().sum(axis = 0)/merged.shape[0]*100

### Data Cleaning

The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

In [ ]:
merged.info()

In [ ]:
def norm(df_,col):
    mincol = df_[col].min()
    maxcol = df_[col].max()
    newcolname = f'{col}_norm'
    df_[newcolname] = (df_[col] - mincol)/(maxcol - mincol)

In [ ]:
norm(merged, 'Distance(mi)')

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder


In [ ]:
merged.columns

In [ ]:
oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(merged[["Weather_Condition"]])
Weather_Condition_df = pd.DataFrame(oe_results.toarray(), columns=oe_style.get_feature_names())
merged = pd.concat([merged,Weather_Condition_df], axis=1).drop(['Weather_Condition'], axis=1)
merged

In [ ]:
merged.columns

In [ ]:
norm(merged ,'Wind_Speed(mph)' )
norm(merged ,'Visibility(mi)' )

In [ ]:
merged.columns

As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [ ]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
to_take = ['Lat', 'Lng', 'Distance(mi)_norm','Junction'
           ,'Stop','Amenity','Side']

train_df, val_df = train_test_split(merged[merged['Wind_Speed(mph)'].notnull()], test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[to_take]
X_val = X_val[to_take]

classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

Now let's test our classifier on the validation dataset and see the accuracy.

Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [ ]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.drop_duplicates(inplace=True)

#fix date and date time
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['date'] = pd.to_datetime(test_df['timestamp'].dt.date)
test_df['hour'] = test_df['timestamp'].dt.strftime('%H')
test_df['hours_Min'] = test_df['timestamp'].dt.strftime('%H:%M')

# this cell is used to convert T/F features into 0/1 features
test_df['Bump'] = test_df['Bump'].astype(int)
test_df['Crossing'] = test_df['Crossing'].astype(int)
test_df['Give_Way'] = test_df['Give_Way'].astype(int)
test_df['Junction'] = test_df['Junction'].astype(int)
test_df['No_Exit'] = test_df['No_Exit'].astype(int)
test_df['Railway'] = test_df['Railway'].astype(int)
test_df['Roundabout'] = test_df['Roundabout'].astype(int)
test_df['Stop'] = test_df['Stop'].astype(int)
test_df['Amenity'] = test_df['Amenity'].astype(int)
test_df['hour'] = test_df['hour'].astype(int)



# this cell is used to convert the Side column from L/R data into 0/1 data
test_df['Side'] = test_df['Side'].map(dict(L=0, R=1))

test_df.drop(oneValuedcols,axis=1,inplace=True)
test_df.shape

In [ ]:
test_df = pd.merge(test_df,weather_df,how = "left" , left_on=['date','hour'],right_on=['date','hour'] )
# test_df = test_df.merge(holidays_df , how = 'left' , on = 'date')
test_df.shape


In [ ]:
test_df  = test_df[test_df.Weather_Condition.notnull()]
# test_df['description'] = test_df['description'].fillna(0)
# test_df.loc[test_df['description']!= 0, 'description'] = 1
norm(test_df, 'Distance(mi)')
norm(test_df ,'Wind_Speed(mph)' )
norm(test_df ,'Visibility(mi)' )

oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(test_df[["Weather_Condition"]])
Weather_Condition_df = pd.DataFrame(oe_results.toarray(), columns=oe_style.get_feature_names())
test_df = pd.concat([test_df,Weather_Condition_df], axis=1).drop(['Weather_Condition'], axis=1)
test_df.shape

In [ ]:
test_df.columns

Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [ ]:
# test_df = test_df[test_df['Wind_Speed(mph)'].notnull()]

In [ ]:
test_df.shape

In [ ]:
to_take

In [ ]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[to_take]
y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df.head()

Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [ ]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
test_df.shape

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.